In [2]:
import pandas as pd
import seaborn as sb
import numpy as np

In [5]:
import os


def get_tuneBooks_file_names(path):
    tuneBook_filenames = []
    
    for tuneBook_name in os.listdir(path):
        if tuneBook_name.endswith('.abc'):
            tuneBook_filenames.append(tuneBook_name)
    
    #return ["jigs.abc"]
    return tuneBook_filenames
def split_abc_song(abc_song):
    k_index = abc_song.find('K:')

    split_index = abc_song.find('\n', k_index)

    header = abc_song[:split_index]
    body = abc_song[split_index+1:]
    return header, body

def tuneBook_to_dataframe(tuneBook):
    song_list = contents.split("\n\n")
    song_list = [song.strip() for song in song_list]
    songs_header_body_format = [split_abc_song(song) for song in song_list if song != '']
    """
    for (h,b) in songs_header_body_format:
        print("header")
        print(h)
        print("*"*10)
        print("body")
        print(b)
        print("\n")
        print("-"*50)
        print("\n")
    """
    
    return pd.DataFrame(songs_header_body_format, columns=['header', 'body'])

def get_attributes_from_song_header(tuneBook_df,tuneBook_name):
    attributes_column = tuneBook_df["header"]
    attributes_list = list(attributes_column)
    
    data = []
    for string in attributes_list:
        song = {}
        for line in string.split("\n"):
            if ':' in line:
                key, value = line.split(":", maxsplit=1)
                song[key.strip()] = value.strip()
        data.append(song)

    df = pd.DataFrame(data)
    descriptive_names = {
                        'X': 'reference_number',
                        'T': 'title',
                        'S': 'source',
                        'M': 'meter',
                        'L': 'unit_note_length',
                        'R': 'rhythm',
                        'P': 'parts',
                        'K': 'key',
                        'F': 'file_name',
                        'N': 'notes'
                        }
    df.rename(columns=descriptive_names, inplace=True)
    df["tuneBook"] = tuneBook_name
    df["original_header"] =  tuneBook_df["header"]
    df["original_body"] =  tuneBook_df["body"]
    
    return df

### treat strings line by line to ignore comments and parts!!!!!
def get_song_characteristics_from_body(tuneBook_df, header_col_name="original_body"):
    songs_bodies=tuneBook_df[header_col_name]
    tuneBook["num_notes_per_bar"] # count num -> useless? to measure intensity?
    tuneBook["num_bars"] # count number of || ins song -> to detect possible outliers?
    tuneBook["num_notes_in_song"] # count a,b,c,d...in string, match regex "a"   
    tuneBook["chord_progression"] #string "|"C"|"Dm"|" -> to count chords and to have a "Tree view of common chord progressions"
    tuneBook["multiple_parts"] #bool -> to treat songs with multiple parts
    ##  Count types of notes in each song to see the proportion of notes for a given key
    ## Maybe by chord also?
    return []
def get_songs_metadata(songs_dataframe, tuneBook_name):
    #songs_dataframe["number_in_tunebook"] = get_attributes_from_song_header(songs_dataframe)
    df = get_attributes_from_song_header(songs_dataframe,tuneBook_name)
    get_song_characteristics_from_body(df)
    return df

absolute_path = os.getcwd()
relative_path = "notebooks/data/NottinghamData/nottingham_database"
absolute_path = os.path.join(absolute_path, relative_path)
songs_df = pd.DataFrame()
list_tuneBooks = get_tuneBooks_file_names(absolute_path)
for abc_tuneBook_filename in list_tuneBooks:
    file_path = os.path.join(absolute_path, abc_tuneBook_filename)
    
    with open(file_path) as tuneBook:
        contents = tuneBook.read()
        df = tuneBook_to_dataframe(contents)
        df1 = get_songs_metadata(df,abc_tuneBook_filename)
        songs_df = pd.concat([songs_df, df1], ignore_index=True)

#print(songs_df.isna().sum())
#songs_df

reference_number       1
title                  1
source                 2
meter                  1
unit_note_length     396
key                    1
parts                923
%P                  1022
tuneBook               0
original_header        0
original_body          0
notes               1036
rhythm               117
file_name            662
dtype: int64


,reference_number,title,source,meter,unit_note_length,key,parts,%P,tuneBook,original_header,original_body,notes,rhythm,file_name
0,1,The Alderman's Hat,Playford,4/4,1/4,Gm,NaN,NaN,playford.abc,X: 1\nT:The Alderman's Hat\n% Nottingham Music...,"P:A\nD|""Gm""GB2G|""D7""^Fd2D|""Gm""GB2G|""D7""^FA2D|""...",NaN,NaN,NaN
1,2,CAL 1,Mark Knopfler,9/8,1/8,G,AABABA,NaN,playford.abc,X: 2\nT:CAL 1\n% Nottingham Music Database\nS:...,"P:A\n|G2E |""G""D3 -D2E ""f#""G2B|""Em""A3 G3 -""d""G2...",NaN,NaN,NaN
2,3,CAL 2,Mark Knopfler,9/8,NaN,G,A(acc) A(unacc),NaN,playford.abc,X: 3\nT:CAL 2\n% Nottingham Music Database\nS:...,"D2G |""G""B3 -B3 BAG|""G""dB2 -B3 ABd|""G""d3 -d2B ""...",NaN,NaN,NaN
3,4,Dick's Maggot,Playford,3/2,NaN,G,NaN,NaN,playford.abc,X: 4\nT:Dick's Maggot\n% Nottingham Music Data...,"%%MIDI gchord fzczcz\n""G""g2d4B2-""D7""B2A2|""G""Bc...",NaN,NaN,NaN
4,5,Hole in the Wall,Playford,3/2,NaN,Bb,AAB,NaN,playford.abc,X: 5\nT:Hole in the Wall\n% Nottingham Music D...,"%%MIDI gchord fzczcz\nP:A\nB2c2||\\nP:segno\n""...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,27,Winster Processional,Eric Foxley,4/4,1/4,G,NaN,NaN,morris.abc,X: 27\nT:Winster Processional\n% Nottingham Mu...,"d|""G""gd ""D7""de/2f/2|""G""gd ""D7""de/2f/2|""G""gd ""D...",NaN,NaN,NaN
1034,28,Sweet Jenny Jones,Adderbury,3/4,1/4,C,ABA,NaN,morris.abc,X: 28\nT:Sweet Jenny Jones\n% Nottingham Music...,P:A\nG|cGE|C3/2D/2E|FAd|BGB|cGE|F3/2G/2A|GcB|c...,NaN,NaN,NaN
1035,29,Singing of the Travels,Symondsbury,4/4,1/4,G,NaN,NaN,morris.abc,X: 29\nT:Singing of the Travels\n% Nottingham ...,"d3/2d/2 |""G""ed dB|""C""c2 d3/2c/2|""G""cB AG|""Am""G...",NaN,NaN,NaN
1036,30,Valentine,Fieldtown,6/8,NaN,G,A(ABBB)(ACCC)A,NaN,morris.abc,X: 30\nT:Valentine\n% Nottingham Music Databas...,"P:A\ng|""C""gfe ede|""C""edc cBc|""G""dBg dBG|""Am""FA...",NaN,NaN,NaN


In [9]:
songs_df["original_body"].str.contains("P:")

0        True
1        True
2       False
3       False
4        True
        ...  
1033    False
1034     True
1035    False
1036     True
1037     True
Name: original_body, Length: 1038, dtype: bool